# Banco_de_Dados_Educacao

In [1]:
import tkinter as tk
import psycopg2

# Função para conectar ao banco de dados PostgreSQL
def conectar_bd():
    try:
        conexao = psycopg2.connect(
            host="localhost",
            database="postgres",
            user="postgres",
            password="4107"
        )
        return conexao
    except (Exception, psycopg2.Error) as error:
        print("Erro ao conectar ao PostgreSQL", error)
        return None

# Função genérica para executar comandos de inserção
def executar_insercao(query, valores, label_mensagem):
    conexao = conectar_bd()
    if conexao is not None:
        cursor = conexao.cursor()
        try:
            cursor.execute(query, valores)
            conexao.commit()
            label_mensagem.config(text="Dados inseridos com sucesso!")
        except (Exception, psycopg2.Error) as error:
            print("Erro ao inserir dados", error)
            label_mensagem.config(text="Erro ao inserir dados!")
            conexao.rollback()
        finally:
            cursor.close()
            conexao.close()

# Função para carregar e exibir dados dos alunos na Listbox
def atualizar_lista_alunos():
    conexao = conectar_bd()
    if conexao is not None:
        cursor = conexao.cursor()
        try:
            cursor.execute("SELECT * FROM Alunos")
            alunos = cursor.fetchall()
            listbox_alunos.delete(0, tk.END)  # Limpar a Listbox
            for aluno in alunos:
                listbox_alunos.insert(tk.END, aluno)
        except (Exception, psycopg2.Error) as error:
            print("Erro ao consultar dados", error)
        finally:
            cursor.close()
            conexao.close()

# Função para consultar a tabela e exibir resultados em uma nova janela
def consultar_tabela():
    conexao = conectar_bd()
    if conexao is not None:
        cursor = conexao.cursor()
        try:
            query = "SELECT * FROM Alunos;"  # Altere para a tabela desejada
            cursor.execute(query)
            registros = cursor.fetchall()

            # Criar nova janela para exibir os resultados
            janela_consulta = tk.Toplevel(janela)
            janela_consulta.title("Consulta de Alunos")
            janela_consulta.geometry("400x300")

            # Adicionar uma Listbox para exibir os registros
            listbox_consulta = tk.Listbox(janela_consulta, width=50, height=15)
            listbox_consulta.pack(padx=10, pady=10)

            # Inserir registros na Listbox
            for registro in registros:
                listbox_consulta.insert(tk.END, registro)

        except (Exception, psycopg2.Error) as error:
            print("Erro ao consultar a tabela", error)
        finally:
            cursor.close()
            conexao.close()

# Funções para inserir dados nas tabelas
def inserir_dados_aluno():
    nome = entry_nome.get()
    idade = entry_idade.get()
    sexo = entry_sexo.get()
    origem = entry_origem.get()
    query = '''
    INSERT INTO Alunos (nome, idade, sexo, origem)
    VALUES (%s, %s, %s, %s)
    '''
    valores = (nome, idade, sexo, origem)
    executar_insercao(query, valores, label_mensagem_aluno)
    atualizar_lista_alunos()  # Atualiza a lista após inserção
    entry_nome.delete(0, tk.END)
    entry_idade.delete(0, tk.END)
    entry_sexo.delete(0, tk.END)
    entry_origem.delete(0, tk.END)

def inserir_dados_sessao():
    data = entry_data_sessao.get()
    conteudo = entry_conteudo.get()
    query = '''
    INSERT INTO Sessoes (data, conteudo)
    VALUES (%s, %s)
    '''
    valores = (data, conteudo)
    executar_insercao(query, valores, label_mensagem_sessao)
    entry_data_sessao.delete(0, tk.END)
    entry_conteudo.delete(0, tk.END)

def inserir_dados_participacao():
    id_aluno = entry_id_aluno_participacao.get()
    id_sessao = entry_id_sessao_participacao.get()
    presente = entry_presente.get().lower() in ['true', '1', 'sim']
    query = '''
    INSERT INTO Participacao (id_aluno, id_sessao, presente)
    VALUES (%s, %s, %s)
    '''
    valores = (id_aluno, id_sessao, presente)
    executar_insercao(query, valores, label_mensagem_participacao)
    entry_id_aluno_participacao.delete(0, tk.END)
    entry_id_sessao_participacao.delete(0, tk.END)
    entry_presente.delete(0, tk.END)

def inserir_dados_projeto():
    id_aluno = entry_id_aluno_projeto.get()
    descricao = entry_descricao.get()
    data_submissao = entry_data_submissao.get()
    query = '''
    INSERT INTO Projetos (id_aluno, descricao, data_submissao)
    VALUES (%s, %s, %s)
    '''
    valores = (id_aluno, descricao, data_submissao)
    executar_insercao(query, valores, label_mensagem_projeto)
    entry_id_aluno_projeto.delete(0, tk.END)
    entry_descricao.delete(0, tk.END)
    entry_data_submissao.delete(0, tk.END)

def inserir_dados_feedback():
    id_aluno = entry_id_aluno_feedback.get()
    comentario = entry_comentario.get()
    data_feedback = entry_data_feedback.get()
    query = '''
    INSERT INTO Feedbacks (id_aluno, comentario, data_feedback)
    VALUES (%s, %s, %s)
    '''
    valores = (id_aluno, comentario, data_feedback)
    executar_insercao(query, valores, label_mensagem_feedback)
    entry_id_aluno_feedback.delete(0, tk.END)
    entry_comentario.delete(0, tk.END)
    entry_data_feedback.delete(0, tk.END)

# Criar a janela principal
janela = tk.Tk()
janela.title("Controle e Monitoramento de Alunos - Projetos")
janela.geometry("700x665")  # Ajustar largura para acomodar a Listbox

# Adicionar um título na janela
titulo_label = tk.Label(janela, text="Cadastro de Alunos", font=("Helvetica", 16, "bold"))
titulo_label.grid(row=0, column=0, columnspan=2, pady=10, padx=10)

# Seção para inserir dados de Alunos
tk.Label(janela, text="Nome:").grid(row=1, column=0, padx=10)
entry_nome = tk.Entry(janela)
entry_nome.grid(row=1, column=1, padx=10)

tk.Label(janela, text="Idade:").grid(row=2, column=0, padx=10)
entry_idade = tk.Entry(janela)
entry_idade.grid(row=2, column=1, padx=10)

tk.Label(janela, text="Sexo:").grid(row=3, column=0, padx=10)
entry_sexo = tk.Entry(janela)
entry_sexo.grid(row=3, column=1, padx=10)

tk.Label(janela, text="Origem:").grid(row=4, column=0, padx=10)
entry_origem = tk.Entry(janela)
entry_origem.grid(row=4, column=1, padx=10)

botao_inserir_aluno = tk.Button(janela, text="Inserir Aluno", command=inserir_dados_aluno)
botao_inserir_aluno.grid(row=5, column=0, columnspan=2, pady=5)

label_mensagem_aluno = tk.Label(janela, text="")
label_mensagem_aluno.grid(row=6, column=0, columnspan=2)

# Seção para inserir dados de Sessões
tk.Label(janela, text="Data (YYYY-MM-DD):").grid(row=7, column=0, padx=10)
entry_data_sessao = tk.Entry(janela)
entry_data_sessao.grid(row=7, column=1, padx=10)

tk.Label(janela, text="Conteúdo:").grid(row=8, column=0, padx=10)
entry_conteudo = tk.Entry(janela)
entry_conteudo.grid(row=8, column=1, padx=10)

botao_inserir_sessao = tk.Button(janela, text="Inserir Sessão", command=inserir_dados_sessao)
botao_inserir_sessao.grid(row=9, column=0, columnspan=2, pady=5)

label_mensagem_sessao = tk.Label(janela, text="")
label_mensagem_sessao.grid(row=10, column=0, columnspan=2)

# Seção para inserir dados de Participações
tk.Label(janela, text="ID Aluno:").grid(row=11, column=0, padx=10)
entry_id_aluno_participacao = tk.Entry(janela)
entry_id_aluno_participacao.grid(row=11, column=1, padx=10)

tk.Label(janela, text="ID Sessão:").grid(row=12, column=0, padx=10)
entry_id_sessao_participacao = tk.Entry(janela)
entry_id_sessao_participacao.grid(row=12, column=1, padx=10)

tk.Label(janela, text="Presente (True/False):").grid(row=13, column=0, padx=10)
entry_presente = tk.Entry(janela)
entry_presente.grid(row=13, column=1, padx=10)

botao_inserir_participacao = tk.Button(janela, text="Inserir Participação", command=inserir_dados_participacao)
botao_inserir_participacao.grid(row=14, column=0, columnspan=2, pady=5)

label_mensagem_participacao = tk.Label(janela, text="")
label_mensagem_participacao.grid(row=15, column=0, columnspan=2)

# Seção para inserir dados de Projetos
tk.Label(janela, text="ID Aluno:").grid(row=16, column=0, padx=10)
entry_id_aluno_projeto = tk.Entry(janela)
entry_id_aluno_projeto.grid(row=16, column=1, padx=10)

tk.Label(janela, text="Descrição:").grid(row=17, column=0, padx=10)
entry_descricao = tk.Entry(janela)
entry_descricao.grid(row=17, column=1, padx=10)

tk.Label(janela, text="Data de Submissão (YYYY-MM-DD):").grid(row=18, column=0, padx=10)
entry_data_submissao = tk.Entry(janela)
entry_data_submissao.grid(row=18, column=1, padx=10)

botao_inserir_projeto = tk.Button(janela, text="Inserir Projeto", command=inserir_dados_projeto)
botao_inserir_projeto.grid(row=19, column=0, columnspan=2, pady=5)

label_mensagem_projeto = tk.Label(janela, text="")
label_mensagem_projeto.grid(row=20, column=0, columnspan=2)

# Seção para inserir dados de Feedbacks
tk.Label(janela, text="ID Aluno:").grid(row=21, column=0, padx=10)
entry_id_aluno_feedback = tk.Entry(janela)
entry_id_aluno_feedback.grid(row=21, column=1, padx=10)

tk.Label(janela, text="Comentário:").grid(row=22, column=0, padx=10)
entry_comentario = tk.Entry(janela)
entry_comentario.grid(row=22, column=1, padx=10)

tk.Label(janela, text="Data de Feedback (YYYY-MM-DD):").grid(row=23, column=0, padx=10)
entry_data_feedback = tk.Entry(janela)
entry_data_feedback.grid(row=23, column=1, padx=10)

botao_inserir_feedback = tk.Button(janela, text="Inserir Feedback", command=inserir_dados_feedback)
botao_inserir_feedback.grid(row=24, column=0, columnspan=2, pady=5)

label_mensagem_feedback = tk.Label(janela, text="")
label_mensagem_feedback.grid(row=25, column=0, columnspan=2)

# Criar a Listbox para exibir alunos
listbox_alunos = tk.Listbox(janela, width=50, height=15)
listbox_alunos.grid(row=0, column=2, rowspan=20, padx=10, pady=10)

# Atualizar a lista de alunos ao iniciar a aplicação
atualizar_lista_alunos()

# Adicionar campo para consultar dados de um aluno específico
tk.Label(janela, text="Nome do Aluno para Consulta:").grid(row=21, column=2, padx=10)
entry_nome_consulta = tk.Entry(janela)
entry_nome_consulta.grid(row=20, column=2, padx=10)

# Adicionar o botão de consulta baseado no nome do aluno
botao_consultar_aluno = tk.Button(janela, text="Consultar Aluno", command=lambda: consultar_aluno(entry_nome_consulta.get()))
botao_consultar_aluno.grid(row=22, column=2, columnspan=2, pady=10)

# Adicionar o botão de consulta
#botao_consultar = tk.Button(janela, text="Consultar Dados", command=consultar_tabela)
#botao_consultar.grid(row=20, column=2, pady=10)

# Iniciar o loop principal da interface gráfica
janela.mainloop()